# 03 — Value Scoring

Compute composite value scores for every player-season:
- Position-specific performance z-scores
- Salary z-scores within position
- Value Score = performance_z - salary_z
  - Positive = bargain (outperforming salary)
  - Negative = overpaid (underperforming salary)

In [1]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
from src.data_loader import DATA_DIR
from src.value_score import (
    compute_value_scores, top_bargains, top_overpaid,
    POSITION_WEIGHTS
)

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 200)
pd.set_option('display.float_format', '{:.3f}'.format)

E:\Python\Python38-32\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
E:\Python\Python38-32\lib\site-packages\numpy\.libs\libopenblas.D6ALFJ4QQDWP6YNOQJNPYL27LRE6SILT.gfortran-win32.dll
E:\Python\Python38-32\lib\site-packages\numpy\.libs\libopenblas_v0.3.21-gcc_8_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Load Analysis-Ready Data

In [2]:
df = pd.read_parquet(DATA_DIR / 'analysis_ready.parquet', engine='fastparquet')
print(f"Loaded: {df.shape}")
print(f"Position groups: {sorted(df['pos_group'].unique())}")
print(f"\nRows per position:")
print(df['pos_group'].value_counts().to_string())

Loaded: (9140, 73)
Position groups: ['DB', 'DL', 'K', 'LB', 'OL', 'P', 'QB', 'RB', 'TE', 'WR']

Rows per position:
pos_group
DB    1953
WR    1792
DL    1257
RB    1249
LB    1126
TE    1063
QB     497
P       99
OL      92
K       12


## Position Weight Configuration

In [3]:
print("Stat weights by position group:")
for pos, weights in POSITION_WEIGHTS.items():
    print(f"\n{pos}:")
    for stat, w in weights.items():
        available = stat in df.columns or stat.endswith('_inv')
        marker = '✓' if available else '✗'
        print(f"  {marker} {stat:25s} {w:.2f}")

Stat weights by position group:

QB:
  ✓ passing_yards             0.20
  ✓ passing_tds               0.25
  ✓ int_rate_inv              0.15
  ✓ passer_rating             0.20
  ✓ completion_pct            0.10
  ✓ rushing_yards             0.10

RB:
  ✓ rushing_yards             0.30
  ✓ rushing_tds               0.25
  ✓ yards_per_carry           0.15
  ✓ receiving_yards           0.20
  ✓ fumbles_inv               0.10

WR:
  ✓ receiving_yards           0.30
  ✓ receptions                0.20
  ✓ receiving_tds             0.30
  ✓ catch_rate                0.20

TE:
  ✓ receiving_yards           0.30
  ✓ receptions                0.20
  ✓ receiving_tds             0.30
  ✓ catch_rate                0.20

OL:
  ✓ total_snaps               0.70
  ✓ games_played              0.30

DL:
  ✓ def_sacks                 0.35
  ✓ def_qb_hits               0.25
  ✓ def_tackles               0.25
  ✓ def_pressures             0.15

LB:
  ✓ def_tackles               0.25
  ✓ def_sacks          

## Compute Value Scores

In [4]:
scored = compute_value_scores(df)

print(f"Scored dataset: {scored.shape}")
print(f"\nValue score stats by position:")
print(scored.groupby('pos_group')['value_score'].describe().round(2).to_string())

Scored dataset: (9041, 79)

Value score stats by position:
             count   mean   std    min    25%    50%   75%   max
pos_group                                                       
DB        1827.000 -0.010 1.040 -5.570 -0.400 -0.040 0.530 3.400
DL        1242.000  0.010 0.860 -4.510 -0.280 -0.010 0.410 4.310
K           12.000  0.000 1.000 -1.370 -0.820 -0.210 1.060 1.160
LB        1089.000 -0.000 0.900 -6.360 -0.230 -0.010 0.380 2.900
OL          92.000  0.000 1.010 -3.670 -0.410  0.130 0.580 2.300
QB         495.000 -0.000 0.890 -2.630 -0.550  0.040 0.610 2.270
RB        1195.000  0.010 0.950 -4.890 -0.290  0.090 0.540 2.570
TE        1060.000 -0.000 0.860 -3.460 -0.390  0.080 0.440 3.290
WR        1782.000 -0.000 0.900 -3.870 -0.400  0.020 0.460 3.120


## Top Bargains — All Positions

In [5]:
print("=== TOP 15 BARGAINS (ALL POSITIONS) ===")
top_bargains(scored, n=15)

=== TOP 15 BARGAINS (ALL POSITIONS) ===


,player_name,pos_group,recent_team,season,apy_cap_pct,performance_zscore,salary_zscore,value_score
7238,Nick Bosa,DL,SF,2022,0.045,5.529,1.224,4.306
7491,Justin Madubuike,DL,BAL,2023,0.006,3.525,-0.428,3.953
2209,Kerby Joseph,DB,DET,2024,0.006,3.010,-0.394,3.404
4492,Jordan Reed,TE,WAS,2015,0.006,2.774,-0.514,3.288
7243,Dexter Lawrence,DL,NYG,2022,0.018,3.226,0.080,3.145
4041,Amon-Ra St. Brown,WR,DET,2023,0.006,2.612,-0.509,3.121
1921,DaRon Bland,DB,DAL,2023,0.005,2.624,-0.449,3.073
7492,Aidan Hutchinson,DL,DET,2023,0.043,4.150,1.139,3.011
3227,Michael Thomas,WR,NO,2018,0.008,2.553,-0.439,2.992
5280,Sam LaPorta,TE,DET,2023,0.011,2.726,-0.235,2.961


## Top Overpaid — All Positions

In [6]:
print("=== TOP 15 OVERPAID (ALL POSITIONS) ===")
top_overpaid(scored, n=15)

=== TOP 15 OVERPAID (ALL POSITIONS) ===


,player_name,pos_group,recent_team,season,apy_cap_pct,performance_zscore,salary_zscore,value_score
8597,Joey Bosa,LB,LAC,2022,0.136,-0.308,6.054,-6.362
8515,TJ Watt,LB,PIT,2022,0.153,1.165,6.889,-5.724
8778,Joey Bosa,LB,LAC,2023,0.136,0.396,6.054,-5.658
2307,Marshon Lattimore,DB,WAS,2024,0.106,-0.503,5.063,-5.566
1751,Marshon Lattimore,DB,NO,2022,0.106,-0.297,5.063,-5.361
2306,Marshon Lattimore,DB,NO,2024,0.106,-0.217,5.063,-5.280
2305,Marshon Lattimore,DB,2TM,2024,0.106,-0.184,5.063,-5.247
2049,Jaire Alexander,DB,GB,2023,0.101,-0.416,4.790,-5.207
1972,Marshon Lattimore,DB,NO,2023,0.106,0.047,5.063,-5.016
2364,Jaire Alexander,DB,GB,2024,0.101,-0.198,4.790,-4.989


## Position-by-Position Breakdown

In [7]:
for pos in ['QB', 'WR', 'RB', 'TE', 'DL', 'LB', 'DB']:
    print(f"\n{'='*60}")
    print(f"  {pos} — Top 5 Bargains")
    print(f"{'='*60}")
    display(top_bargains(scored, pos_group=pos, n=5))
    
    print(f"\n  {pos} — Top 5 Overpaid")
    display(top_overpaid(scored, pos_group=pos, n=5))


  QB — Top 5 Bargains


,player_name,pos_group,recent_team,season,apy_cap_pct,performance_zscore,salary_zscore,value_score
355,Lamar Jackson,QB,BAL,2019,0.013,1.482,-0.789,2.272
325,Patrick Mahomes,QB,KC,2018,0.025,1.644,-0.625,2.269
369,Josh Allen,QB,BUF,2020,0.030,1.501,-0.557,2.058
465,Brock Purdy,QB,SF,2023,0.004,1.106,-0.912,2.019
434,Jalen Hurts,QB,PHI,2022,0.008,1.147,-0.857,2.004



  QB — Top 5 Overpaid


,player_name,pos_group,recent_team,season,apy_cap_pct,performance_zscore,salary_zscore,value_score
312,Deshaun Watson,QB,CLE,2022,0.221,-0.576,2.050,-2.626
109,Joe Webb,QB,BUF,2017,0.005,-3.478,-0.898,-2.579
280,Dak Prescott,QB,DAL,2024,0.235,-0.224,2.241,-2.465
313,Deshaun Watson,QB,CLE,2023,0.221,-0.406,2.050,-2.456
314,Deshaun Watson,QB,CLE,2024,0.221,-0.377,2.050,-2.428



  WR — Top 5 Bargains


...

,player_name,pos_group,recent_team,season,apy_cap_pct,performance_zscore,salary_zscore,value_score
8597,Joey Bosa,LB,LAC,2022,0.136,-0.308,6.054,-6.362
8515,TJ Watt,LB,PIT,2022,0.153,1.165,6.889,-5.724
8778,Joey Bosa,LB,LAC,2023,0.136,0.396,6.054,-5.658
8497,Khalil Mack,LB,LAC,2022,0.133,1.112,5.906,-4.794
8853,Von Miller,LB,BUF,2023,0.096,-0.540,4.089,-4.629



  DB — Top 5 Bargains


,player_name,pos_group,recent_team,season,apy_cap_pct,performance_zscore,salary_zscore,value_score
2209,Kerby Joseph,DB,DET,2024,0.006,3.010,-0.394,3.404
1921,DaRon Bland,DB,DAL,2023,0.005,2.624,-0.449,3.073
1559,LJarius Sneed,DB,KC,2022,0.005,2.328,-0.449,2.776
1875,Antoine Winfield,DB,TB,2023,0.009,2.538,-0.230,2.769
649,Damontae Kazee,DB,ATL,2018,0.004,2.197,-0.503,2.701



  DB — Top 5 Overpaid


,player_name,pos_group,recent_team,season,apy_cap_pct,performance_zscore,salary_zscore,value_score
2307,Marshon Lattimore,DB,WAS,2024,0.106,-0.503,5.063,-5.566
1751,Marshon Lattimore,DB,NO,2022,0.106,-0.297,5.063,-5.361
2306,Marshon Lattimore,DB,NO,2024,0.106,-0.217,5.063,-5.280
2305,Marshon Lattimore,DB,2TM,2024,0.106,-0.184,5.063,-5.247
2049,Jaire Alexander,DB,GB,2023,0.101,-0.416,4.790,-5.207


## Spot-Check: Known Players

In [8]:
spot_checks = ['Mahomes', 'Allen', 'Donald', 'Kelce', 'Henry', 'Jefferson']

for name in spot_checks:
    player = scored[scored['player_name'].str.contains(name, case=False, na=False)]
    if len(player) > 0:
        latest = player.sort_values('season').iloc[-1]
        print(f"{latest['player_name']:25s} ({latest['pos_group']}, {int(latest['season'])}) | "
              f"Perf Z: {latest['performance_zscore']:+.2f} | "
              f"Salary Z: {latest['salary_zscore']:+.2f} | "
              f"Value: {latest['value_score']:+.2f} | "
              f"Cap%: {latest['apy_cap_pct']:.1%}")
    else:
        print(f"{name}: not found")

Patrick Mahomes           (QB, 2024) | Perf Z: +0.91 | Salary Z: +2.13 | Value: -1.22 | Cap%: 22.7%
Davis Allen               (TE, 2024) | Perf Z: -0.73 | Salary Z: -0.57 | Value: -0.16 | Cap%: 0.5%
Donald Parham             (TE, 2023) | Perf Z: +0.28 | Salary Z: -0.51 | Value: +0.80 | Cap%: 0.6%
Travis Kelce              (TE, 2024) | Perf Z: +1.81 | Salary Z: +2.89 | Value: -1.09 | Cap%: 6.7%
Hunter Henry              (TE, 2024) | Perf Z: +0.79 | Salary Z: +1.11 | Value: -0.31 | Cap%: 3.5%
Jordan Jefferson          (DL, 2024) | Perf Z: -0.62 | Salary Z: -0.47 | Value: -0.15 | Cap%: 0.5%


## Outlier Summary

In [9]:
print("Outlier counts by position:")
outlier_summary = scored.groupby('pos_group').agg(
    total=('value_score', 'count'),
    bargains=('is_bargain', 'sum'),
    overpaid=('is_overpaid', 'sum'),
).astype(int)
outlier_summary['bargain_pct'] = (outlier_summary['bargains'] / outlier_summary['total'] * 100).round(1)
outlier_summary['overpaid_pct'] = (outlier_summary['overpaid'] / outlier_summary['total'] * 100).round(1)
print(outlier_summary.to_string())

Outlier counts by position:
           total  bargains  overpaid  bargain_pct  overpaid_pct
pos_group                                                      
DB          1827        33        72        1.800         3.900
DL          1242        15        39        1.200         3.100
K             12         0         0        0.000         0.000
LB          1089         8        44        0.700         4.000
OL            92         1         5        1.100         5.400
QB           495         5        10        1.000         2.000
RB          1195        10        50        0.800         4.200
TE          1060        18        31        1.700         2.900
WR          1782        30        53        1.700         3.000


## Save Scored Dataset

In [10]:
output_path = DATA_DIR / 'scored.parquet'
scored.to_parquet(output_path, engine='fastparquet', index=False)
print(f"Saved scored data: {output_path}")
print(f"  {scored.shape[0]:,} player-seasons x {scored.shape[1]} columns")

Saved scored data: G:\ai\nfl\data\scored.parquet
  9,041 player-seasons x 79 columns
